In [1]:
from numpythia import Pythia, hepmc_write, hepmc_read
from numpythia import STATUS, HAS_END_VERTEX, ABS_PDG_ID

from pyjet import cluster
from pyjet.testdata import get_event

In [2]:
params = {"Beams:eCM": 13000, "WeakSingleBoson:ffbar2gmZ" : "on", "23:onMode": "off", "23:onIfAny": "13", "WeakZ0:gmZmode": 2}
pythia = Pythia(params =params)

selection = ((STATUS == 1) & ~HAS_END_VERTEX)

for event in pythia(events=1):
    array = event.all(selection)
    muplus = array[array["pdgid"] == 13]

In [3]:
vectors = get_event()
sequence = cluster(vectors, R=1.0, p =-1)
jets = sequence.inclusive_jets() # list of PseudoJets